In [6]:
# you may need to `pip install upgrade transformers`
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM
import torch


PyTorch version:  2.1.1+cpu
Transformers version:  4.35.0


In [3]:
# this will downlaod the model from huggingface.co and cache locally
# run this block once to download the model

# https://huggingface.co/AI-Sweden-Models/gpt-sw3-126m
model_name = 'AI-Sweden-Models/gpt-sw3-126m-instruct'

# download the model and tokenizer from huggingface.co and cache locally
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# save the model and tokenizer locally
MODEL_PATH = "gptsw3model126m"
model.save_pretrained(MODEL_PATH)
tokenizer.save_pretrained(MODEL_PATH)


In [4]:
# load the model and tokenizer from local cache

model = AutoModelForCausalLM.from_pretrained(MODEL_PATH)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

model.eval()

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model.to(device)


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(64000, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): GELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=64000, bias=False)
)

In [5]:

promptInput = "vad är meningen med livet?"

# this will be different for each model
prompt = f"""
<|endoftext|><s>
User:
{promptInput}

<s>
Bot:
""".strip()


input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to(device)

generated_token_ids = model.generate(
    inputs=input_ids,
    max_new_tokens=100,
    do_sample=True,
    temperature=0.6,
    top_p=1,
)[0]

generated_text = tokenizer.decode(generated_token_ids)    



# code to extract text after "Bot:"
bot_text_start = generated_text.find("Bot:")
if bot_text_start != -1:
    bot_text = generated_text[bot_text_start + len("Bot:"):].strip()
    # stop after <s>
    bot_text = bot_text.split("<s>")[0].strip()
    print(bot_text)
else:
    print("Bot response not found in the generated text.")


Som språkmodell är jag inte en AI, men jag kan ge dig en mängd information om livet och dess betydelse. För att få en bättre förståelse av livet, kan jag ge dig en kort beskrivning av livet och dess betydelse.

Livet är ett komplext och mångfacetterat tillstånd som involverar många aspekter. Det är ett komplext och mångfacetterat tillstånd som involverar många aspekter, inklusive familj, utbildning, ekonomi, relationer, och så vidare. Om du vill förstå
